In [ ]:
import numpy as np
from jax import numpy as jnp
from jax import grad, hessian, jacobian
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
N = 1000
d = 380
x = np.random.randn(N, d)/np.sqrt(d)
w0 = np.random.randn(d)

y = np.sign(x@w0 + 1*np.random.randn(N))

In [ ]:
from sklearn.svm import LinearSVC as SVM
svm = SVM(penalty='l2', loss='squared_hinge', dual=False, fit_intercept=False, C=1e10)
svm = svm.fit(x, y)

In [ ]:
w = svm.coef_.ravel()

In [ ]:
plt.scatter(w, w0)

In [ ]:
force = lambda w: jnp.maximum(0, 1 - y*(x@w))
U = lambda w: jnp.sum(1/2*force(w)**2)

In [ ]:
U(w)

In [ ]:
Hu = hessian(U)(w)

In [ ]:
eigs = np.linalg.eigvalsh(Hu)
h=plt.hist(np.sign(eigs)*np.sqrt(np.abs(eigs)), 100)
# vals, bins = np.histogram(np.sign(eigs)*np.sqrt(np.abs(eigs)), 100)
# plt.plot(bins[:-1], vals)
# plt.axvline(0)
plt.ylim(0, 20)

In [ ]:
jac_del = jacobian(force)(w)
H0 = jnp.einsum('ni,nj->ij', jac_del, jac_del)

In [ ]:
Hp = (Hu - H0)
eigs = np.linalg.eigvalsh(Hp)
eigs = eigs[eigs != 0]
h=plt.hist(np.sign(eigs)/np.sqrt(np.abs(eigs)), 100)
plt.ylim(0, 30)

In [ ]:
# h = plt.hist(np.linalg.eigvalsh(Hu), 100)
# h = plt.hist(np.linalg.eigvalsh(H0), 100, alpha=.5)

In [ ]:
jacobian(jacobian(force))(w)

In [ ]:
hes_del = hessian(force)(w)
Hp = jnp.einsum('n,nij->ij', force(w), hes_del)

In [ ]:
eigs = np.linalg.eigvalsh(Hp)
# h=plt.hist(np.sign(eigs)/np.sqrt(np.abs(eigs)), 100)
# plt.ylim(0, 30)

In [ ]:
hes_del